# 检索组件优化与进阶技术
---

## 1. 环境准备
- 安装依赖包
- 设置镜像源

In [ ]:
# 设置国内镜像源（加速下载）
import os

from langchain_core.output_parsers import StrOutputParser

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# 安装依赖包（取消注释运行）
# !pip install langchain faiss-cpu huggingface-hub dashscope PyPDF2

## 2. 文档处理
- 加载PDF文件
- 检查加载结果

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

def load_documents(pdf_paths):
    """加载pdf文档并返回LangChain Document对象列表"""
    all_docs = []
    for path in pdf_paths:
        try:
            loader = PyPDFLoader(path)
            docs = loader.load()
            all_docs.extend(docs)
            print(f"成功加载：{path}（共{len(docs)}页）")
        except Exception as e:
            print(f"加载失败{path}：{str(e)}")
    return all_docs

# 示例使用
pdf_paths = ["pdf_China/中国产品质量法.pdf"]
processed_documents = load_documents(pdf_paths) # 原始文件
print(processed_documents)

## 3. 文本分割与向量化

- 中文文本分割

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(docs):
    """执行文本分割"""
    if not docs:
        raise ValueError("输入文档列表为空")

    # 中文优化分割器
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,     # 每个文本块500字符
        chunk_overlap=100,  # 块间重叠100字符
        separators=["\n\n", "\n", "。", "！", "？"] # 中文分隔符
    )

    # 执行分割
    split_docs = text_splitter.split_documents(docs)
    print(f"分割为{len(split_docs)}个文本块")

    # 查看前2个分割样例
    for i, doc in enumerate(split_docs[:2]):
        print(f"\n块{i+1}:\n{doc.page_content[:200]}...")

    return split_docs

# 示例使用
split_decs = split_documents(processed_documents)

- 文本分割优化

In [ ]:
# # 改进的分割策略
# from langchain_experimental.text_splitter import SemanticChunker
# from langchain.embeddings import OpenAIEmbeddings
#
# def advanced_text_split(docs):
#     embeddings = OpenAIEmbeddings()
#     splitter = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
#
# # 示例使用
# split_decs2 = advanced_text_split(processed_documents)
# print(split_decs2)

- 构建向量数据库

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

def create_vector_store(split_docs):
    """创建并返回向量数据库"""
    if not split_docs:
        raise ValueError("无法用空文档创建向量库")

    embeddings = HuggingFaceEmbeddings(
        model_name="distiluse-base-multilingual-cased-v2"
    )

    vector_store = FAISS.from_documents(split_docs, embeddings)
    print(f"向量库已构建（包含{vector_store.index.ntotal}）个向量")

    return vector_store

# 示例使用
vector_store = create_vector_store(split_decs)

- 检查向量数据库中的基本信息

In [ ]:
print(f"向量数据库类型：{type(vector_store)}")

print(f"向量数据库中文档数量：{vector_store.index.ntotal}")

print(f"向量维度：{vector_store.index.d}")

## 4. 分析当前检索系统的组成

- **向量检索**：目前使用的是FAISS + HuggingFace嵌入模型(distiluse-base-multilingual-cased-v2)
- **文本分割**：RecursiveCharacterTextSplitter以500字符为单位进行分割
- **检索参数**：top-k=3的相似度检索

## 5. 基础实验与性能评估

In [ ]:
import numpy as np
# 检索分析实验
def analyze_retrieval(query, vector_store, top_k=3):
    """深入分析实验结果"""
    # 获取原始检索结果（这句是集成的功能，下面的才是将其分开的细节）
    docs = vector_store.similarity_search(query, k=top_k)

    # 计算相似度分数
    embeddings = vector_store.embedding_function # 返回嵌入模型对象（vector_store没有其他用）
    query_embedding = embeddings.embed_query(query) # 调用嵌入模型对问题编码

    print(query_embedding) # 输出问题的向量

    print(f"\n查询：'{query}'")
    print(f"检索到的{len(docs)}个文档：")

    for i, doc in enumerate(docs[:top_k]):
        doc_embedding = embeddings.embed_query(doc.page_content) # 调用嵌入模型对检索到的文本编码
        similarity = np.dot(query_embedding, doc_embedding) # 计算查询向量和文档向量之间的余弦相似度
        print(f"\n📄 文档{i+1} (相似度: {similarity:.4f}):")
        print(f"  来源: {doc.metadata.get('source', '未知')}")
        print(f"  内容: {doc.page_content[:50]}...")

# 示例使用
queries = [
    "产品质量监督抽查的规定",
    "产品缺陷的法律定义",
    "违反产品质量法的处罚措施"
]

for query in queries:
    analyze_retrieval(query, vector_store)


## 6. 检索质量评估指标

**建立简单的评估体系**：
- 召回率(Recall)：相关文档被检索到的比例
- 准确率(Precision)：检索结果中相关文档的比例
- 平均倒数排名(MRR)：第一个相关文档排名的倒数

In [ ]:
# 简易评估框架
def evaluate_retrieval(query, relevant_doc_indices, vector_store, top_k=3):
    """评估单个查询的检索结果"""
    # 检索到的文档信息
    docs = vector_store.similarity_search(query, k=top_k)
    # 提取检索到的页码
    retrieved_indices = [doc.metadata.get('page', -1) for doc in docs]

    print("docs类型：", type(docs))
    # print(docs)

    print("retrieved_indices类型：", type(retrieved_indices))
    print("检索到的文档页码：", retrieved_indices)

    # 计算指标
    # 检索到的又相关的 = 检索到的 & 相关的
    relevant_retrieved = len(set(retrieved_indices) & set(relevant_doc_indices))

    # 准确率 = 检索到的相关文档数/检索到的文档总数
    precision = relevant_retrieved / top_k

    # 召回率 = 检索到的相关文档数 / 所有相关文档数
    recall = relevant_retrieved / len(relevant_doc_indices)

    # 平均倒数排名（在检索到的文档中，第一个相关文档排名的倒数）（省略了多次查询求平均的步骤）
    try:
        first_relevant_doc = min([i+1 for i, idx in enumerate(retrieved_indices)
                                  if idx in relevant_doc_indices])
        mrr = 1 / first_relevant_doc
    except :
        mrr = 0

    # 返回一个字典
    return {
        'query': query,
        'precision': precision,
        'recall': recall,
        'mrr': mrr
    }

# 示例评估
evaluation_results = []
evaluation_results.append(evaluate_retrieval(
    "产品质量监督抽查的规定",
    [1, 2, 3], # 假设这些页码包含相关内容
    vector_store
))
print(evaluation_results)